# 检索机器人

## Step1 读取faq数据

In [1]:
import pandas as pd

data = pd.read_csv("./data/law_faq.csv")
data.head()

,title,reply
0,在法律中定金与订金的区别订金和定金哪个受,“定金”是指当事人约定由一方向对方给付的，作为债权担保的一定数额的货币，它属于一种法律上的担...
1,盗窃罪的犯罪客体是什么，盗窃罪的犯罪主体,盗窃罪的客体要件本罪侵犯的客体是公私财物的所有权。侵犯的对象，是国家、集体或个人的财物，一般...
2,非法微整形机构构成非法经营罪吗,符合要件就有可能。非法经营罪，是指未经许可经营专营、专卖物品或其他限制买卖的物品，买卖进出口...
3,入室持刀行凶伤人能不能判刑,对于入室持刀伤人涉嫌故意伤害刑事犯罪，一经定罪，故意伤害他人身体的，处三年以下有期徒刑、拘役...
4,对交通事故责任认定书不服怎么办，交通事故损,事故认定书下发后，如果你对认定不满意，可在接到认定书3日内到上一级公安机关复议。


## Step2 加载模型

In [5]:
from dual_model_bert import DualModel

# 需要完成前置模型训练
dual_model = DualModel.from_pretrained("./dual_model/checkpoint-1000/")
dual_model = dual_model.cuda()
dual_model.eval()
print("匹配模型加载成功！")

d:\anaconda3\envs\py39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\anaconda3\envs\py39\lib\site-packages\transformers\modeling_utils.py:463: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend

匹配模型加载成功！


In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("D:/pretrained_model/models--hfl--chinese-macbert-base")

## Step3 将问题编码为向量

In [ ]:
import torch

questions = data['title'].to_list()
vectors = []

with torch.inference_mode():
    for i in range(0, len(questions), 4):
        batch_sens = questions[i : i + 4]
        inputs = tokenizer(batch_sens, return_tensors='pt', max_length=128, truncation=True, padding=True)
        inputs = {k:v.to(dual_model.device) for k,v in inputs.items()}
        vector = dual_model.bert(**inputs)[1] # 拿到的是CLS token向量
        vectors.append(vector)

vectors = torch.concat(vectors, dim=0).cpu().numpy() # Faiss 的 API 设计时假设输入数据是 NumPy 数组，因此为了保持一致性和避免潜在的问题，通常需要将 PyTorch 张量转换为 NumPy 数组。


## Step4 创建索引

## Step5 对问题进行向量编码

## Step6 向量匹配(召回)

## Step7 加载交互模型

## Step8 最终预测(排序)